<a href="https://colab.research.google.com/github/rinnibhan/octopi-psf-visualization/blob/main/PSF_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PSF Visualization 
### Goal: 
  Acquire and visualize PSF function for objective lens on Octopi
### Steps:
1. Import images from GCS
2. Identify bead locations

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive

ROOT = '/content/drive'     # default for the drive
drive.mount(ROOT)           # we mount the drive at /content/drive

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
import glob
import random
import cv2
import os
import re
from skimage import color
from math import sqrt
from skimage import data
from skimage.feature import blob_log
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# remove default colab files
!rm -rf sample_data
!rm adc.json
!mkdir psf_im
%cd psf_im

rm: cannot remove 'adc.json': No such file or directory
/content/psf_im


## Set up GCS

In [ ]:
# global variables
project_id = 'soe-octopi'
bucket_name = 'octopi-malaria-data'

# enter the name of the folder in GCS directly above the folders with the objective configurations' z-stacks
# note: the folder set-up should be: 
# gs_folder_path > "objective config folder" > "0" > *z-stack images* and "blobs" > *PSF images*
gs_folder_path_1 = 'gs://octopi-malaria-data/20210809_20x_beads/'
gs_folder_path_2 = 'gs://octopi-malaria-data/20210310_objectives_characterizations_with_beads/*/'

# accesses all z stack images
gen_folder_path = gs_folder_path_2 + "*/*" 

# place to save random PSFs in drive, with the MIP visualization
dr_save_path = '/content/drive/MyDrive/Sophomore/Research/Octopi/PSF Visualization'

folder_paths_dict = {}

In [ ]:
# set project
def set_project(proj_id):
  !gcloud config set project {proj_id}

# list elements of folder in bucket
def list_folder(f_path):
  files = !gsutil ls "{f_path}"
  return files

# copy folder from bucket to colab; returns name of file saved locally
def copy_single_file(f_path):
  slashes = [m.start() for m in re.finditer(r"/",f_path)]
  location = "./" + f_path[slashes[-3]+1:slashes[-2]] + f_path[slashes[-1]:] # name of path to file saved locally
  folder_paths_dict["./" + f_path[slashes[-3]+1:slashes[-2]]] = f_path[:slashes[-1]]
  !gsutil -m cp -r "{f_path}" "{location}"
  return location

def copy_file_list(files):
  # files: list of GCS paths to files to copy
  local_files = []
  for file in files: 
    location = copy_single_file(file) # saves each file locally
    local_files.append(location)
  local_files = np.asarray(local_files) # names of paths to files saved locally
  return local_files

# list fluorescence images in folder 
def find_fluorescence_images(f_path):
  files = list_folder(f_path)
  files = np.asarray(files)
  find_fluorescence_mask = np.char.find(files, "405")

  files = files[find_fluorescence_mask > 0]
  return files

In [ ]:
# copy all fluorescence images in all folders from gen_folder path
files = find_fluorescence_images(gen_folder_path) # get list of files to copy; fluorescent images from each folder in bucket
copy_file_list(files) # copy files locally; update dict of local file names

Copying gs://octopi-malaria-data/20210310_objectives_characterizations_with_beads/beads #3/10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0/0_0_0Fluorescence 405 nm Ex.bmp...
\ [1/1 files][ 25.8 MiB/ 25.8 MiB] 100% Done                                    
Operation completed over 1 objects/25.8 MiB.                                     
Copying gs://octopi-malaria-data/20210310_objectives_characterizations_with_beads/beads #3/10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0/0_0_10Fluorescence 405 nm Ex.bmp...
\ [1/1 files][ 25.8 MiB/ 25.8 MiB] 100% Done                                    
Operation completed over 1 objects/25.8 MiB.                                     
Copying gs://octopi-malaria-data/20210310_objectives_characterizations_with_beads/beads #3/10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0/0_0_11Fluorescence 405 nm Ex.bmp...
\ [1/1 files][ 25.8 MiB/ 25.8 MiB] 100% Done          

array(['./10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0_0_0Fluorescence 405 nm Ex.bmp',
       './10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0_0_10Fluorescence 405 nm Ex.bmp',
       './10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0_0_11Fluorescence 405 nm Ex.bmp',
       './10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0_0_12Fluorescence 405 nm Ex.bmp',
       './10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0_0_13Fluorescence 405 nm Ex.bmp',
       './10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0_0_14Fluorescence 405 nm Ex.bmp',
       './10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0_0_15Fluorescence 405 nm Ex.bmp',
       './10x  0.3 0.17 coverslip, dz = 6 um, N = 19, take 2_2021-03-10 21-10-42.936615/0_0_16Fluorescence 405 nm Ex.bmp',
       './10x  0.

## Find Maximum Intensity Projection

Steps:
1. Fn: get z-stack 3D HSV array from folder name
2. Fn: get MIP from z-stack 3D HSV array
3. Fn: get blob coordinates + radii from MIP
4. Fn: given blob cross section and destination folder name, save image to GCS
5. Fn: Given blob coordinates + radii and z-stack 3D HSV array + folder name, construct list of 2D arrays of blob cross sections (across all layers of stack); store plots of every blob in this list in a folder (blob_plots) within the folder; also add list to a dictionary (folder name : list)

In [ ]:
# find z-stack of images given folder name
def find_hsv_z_stack(fold_name):
  # fold_name: name of folder saved in psf_im with the stack of images
  im_files_doubled = !ls {fold_name} # named as such because !ls saves files to list in pairs concatenated in 1 string
  im_files_v = [] # concatenated stack of images (each array element = pixel "value", from HSV)
  im_files = [] # list of image file names
  for pair in im_files_doubled: # separate into actual list of files in folder
    comp = pair.split("'")
    im_files.append(comp[1])
    if len(comp) >= 4:
      im_files.append(comp[3])
  for im_path in im_files: # concatenate images into list
    full_path = '/content/psf_im/' + fold_name[1:-1] + '/' + im_path
    try:
      img = Image.open(full_path) # open the image file
      img.verify() # verify that it is, in fact an image
    except (IOError, SyntaxError) as e:
      print('Bad file:', full_path)
      os.remove(full_path)
      continue
    im = imageio.imread(full_path)
    im_hsv = color.rgb2hsv(im) # convert RGB image to HSV
    im_v = im_hsv[:,:,2] # isolate V array
    im_files_v.append(im_v) # add to list of V arrays for each image
  im_z_stack = np.stack(im_files_v) # stack V arrays for all image
  return im_z_stack

# find rgb z-stack of images given folder name (ordered by img file name)
def find_rgb_z_stack(fold_name):
  # fold_name: name of folder saved in psf_im with the stack of images
  im_files_doubled = !ls {fold_name} # named as such because !ls saves files to list in pairs concatenated in 1 string
  im_files = [] # list of image file names
  for pair in im_files_doubled: # separate into actual list of files in folder
    comp = pair.split("'")
    im_files.append(comp[1])
    if len(comp) >= 4:
      im_files.append(comp[3])
  im_files_v = [None] * len(im_files) # concatenated stack of images (each array element = pixel "value", from HSV)
  for im_path in im_files: # concatenate images into list
    im = imageio.imread('/content/psf_im/' + fold_name[1:-1] + '/' + im_path)
    im_ind = im_path[4:im_path.find("Fl")]
    im_files_v[int(im_ind)] = im # add to list of V arrays for each image
  im_z_stack = np.stack(im_files_v) # stack V arrays for all image
  return im_z_stack

# find max intensity projection given a z-stack of images
def find_max_int_proj(im_z_stack):
  return np.max(im_z_stack, axis=0)

# find all max intensity projections for all imported folders + create dictionary:
# returns dictionary of max intensity projections for all of the z-stacks of images (for each nested folder)
def find_all_max_int_proj():
  # make sure you're in top folder, which contains all nested folders of z-stacks
  folders = !ls # get list of folders that were just saved
  max_int_proj_dict = {}
  for folder in folders: # go through folders
    im_z_stack = find_hsv_z_stack(folder)
    im_max_int = find_max_int_proj(im_z_stack) # create a max intensity projection for the stack of images in the folder
    max_int_proj_dict[folder] = im_max_int
  return max_int_proj_dict

# display all MIPs saved in dictionary
def disp_all_max_int_proj(max_int_proj_dict):
  ims = list(max_int_proj_dict.values())
  if len(max_int_proj_dict) > 1:   
    # display all max intensity projections
    f,ax=plt.subplots(1,len(max_int_proj_dict))
    for i in range(len(max_int_proj_dict)):
      im = ims[i]
      ax[i].imshow(im)
    plt.tight_layout()
  else:
    plt.imshow(ims[0])

## Blob detection

In [ ]:
# given z-stack, create 3D array with: 
# 0: x coordinates of blobs (from LoG blob detection)
# 1: y coordinates of blobs
# 2: radii of blobs
def find_LoG_blobs(im_z_stack):
  mip = find_max_int_proj(im_z_stack)
  # find blobs
  blobs_log = blob_log(mip, max_sigma=30, num_sigma=10, threshold=.1)
  blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2) # Compute radii in the 3rd column
  np.swapaxes(blobs_log, 0, 1) # move x coord to first; y coord to second axis
  return blobs_log

# subset of np.array (3D, because RGB) given center index and side length
def fixed_size_subset(arr, x, y, size):
    x = int(x)
    y = int(y)
    o, r = np.divmod(size, 2)
    l = (x-(o+r-1)).clip(0)
    u = (y-(o+r-1)).clip(0)
    arr_ = arr[l: x+o+1, u:y+o+1, :]
    out = np.full((size, size, 3), 0, dtype=arr.dtype) # fill cut-off values with 0's
    out[:arr_.shape[0], :arr_.shape[1], :] = arr_
    return out
    
# return blob cross section given z-stack and x,y coordinates + crop size
def find_blob_cross_section(im_z_stack, x, y, crop_s):
  blob_slices = []
  for i in range(im_z_stack.shape[0]):
    im_rgb = im_z_stack[i]
    blob_square = fixed_size_subset(im_rgb, x, y, crop_s)
    blob_slice = blob_square[int(crop_s/2),:,:]
    blob_slices.append(blob_slice)
    stack = np.stack(blob_slices)
  return stack

# save images of a blob cross section given the cs and the destination folder path
def save_blob_im(blob_cs, loc_file, dest_file):
  im = Image.fromarray(blob_cs)
  im.save(loc_file)
  !gsutil cp '{loc_file}' '{dest_file}'
  return

# save images of every blob cross section in a z-stack; returns list of all blob cross sections 
def find_stack_blob_cross_sections(loc_fold, crop_s):
  hsv_z_stack = find_hsv_z_stack(loc_fold)
  rgb_z_stack = find_rgb_z_stack(loc_fold)
  blobs_log = find_LoG_blobs(hsv_z_stack)
  blob_cross_sections = []
  new_fold = loc_fold[1:-1] + "/blobs"
  print(new_fold + " << new fold")
  !mkdir "{new_fold}"
  for blob in blobs_log: # for every blob; identified in the mip image
    x, y, r = blob
    blob_cs = find_blob_cross_section(rgb_z_stack, x, y, crop_s) # find a 2D array of the cross section
    blob_cross_sections.append(blob_cs) # add it to the list
    # paths
    loc_file = loc_fold[1:-1] + "/blobs/" + str(int(x)) + "_" + str(int(y)) + ".png" # name of local file to save this blob CS image
    dest_fold = folder_paths_dict["./" + loc_fold[1:-1]] # name of bucket folder to save this blob CS image
    dest_file = dest_fold + "/blobs/" + str(int(x)) + "_" + str(int(y)) + ".png" # name of bucket file to save this blob CS image
    print(loc_file, dest_file)
    save_blob_im(blob_cs, loc_file, dest_file)
  return blob_cross_sections

# save images of every z-stack's blob CSs
def find_all_stacks_blob_cross_sections(crop_s):
  # make sure you're in top folder, which contains all nested folders of z-stacks
  folders = !ls # get list of folders that were just saved
  for folder in folders: # go through folders
    find_stack_blob_cross_sections(folder, crop_s)
  return

In [ ]:
# only if you want to save PSF visualizations to GCS
find_all_stacks_blob_cross_sections(50)

## Save Random PSF Images to Google Drive (slide deck)
Saves 10 randomly selected PSF images from GCS to Google Drive for each objective lens z-stack folder

In [ ]:
# get dictionary of all MIPs
mip_dict = find_all_max_int_proj()

Bad file: /content/psf_im/10x #2 0.25 0 coverslip, dz = 6 um, N = 26, take 1 - psf bad_2021-03-10 19-13-55.184098/0_0_23Fluorescence 405 nm Ex.bmp
Bad file: /content/psf_im/10x #2 0.25 0 coverslip, dz = 6 um, N = 26, take 1 - psf bad_2021-03-10 19-13-55.184098/0_0_24Fluorescence 405 nm Ex.bmp
Bad file: /content/psf_im/10x #2 0.25 0 coverslip, dz = 6 um, N = 26, take 1 - psf bad_2021-03-10 19-13-55.184098/0_0_25Fluorescence 405 nm Ex.bmp


In [ ]:
%cd '/content/'

/content


In [ ]:
# saves a side-by-side of the PSF visualization for a bead, and an image of the MIP for that z-stack with the specific bead surrounded by a red box
# also saves just the PSF visualization of the bead to gdr_save_path
# gcs_blob_path: path to blob in GCS ("gs://.../y_x.png")
# gdr_blob_path: path where blob is saved in Google Drive ("/content/drive/.../y_x.png")
def save_blob_local_and_global(gcs_blob_path, gdr_blob_path, mip_dict, z_to_x_ratio):
  # save PSF to Google Drive
  !gsutil cp "{gcs_blob_path}" "{gdr_blob_path}"
  # extract y and x coordinates
  y_x_png = gdr_blob_path[gdr_blob_path.rfind('/')+1:]
  y = y_x_png[:y_x_png.find('_')]
  x = y_x_png[y_x_png.find('_')+1:y_x_png.find('.')]
  y = int(y)
  x = int(x)
  print(str(y) + "," + str(x))
  # get folder name (to get MIP)
  fold = gcs_blob_path
  for i in range(3):
    fold = fold[:fold.rfind('/')]
  fold = fold[fold.rfind('/')+1:]
  fold = "'" + fold + "'"
  # create red box around blob in the MIP image
  mip_boxed = np.copy(mip_dict[fold])
  mip_boxed = addBoundingBox(mip_boxed,x,y,25)
  # get PSF of blob
  im_s = mip_boxed.shape[0]
  psf_im = imageio.imread(gdr_blob_path)
  psf_im_width = psf_im.shape[1] # number of columns (width)
  psf_im_height = psf_im.shape[0] * z_to_x_ratio # appropriate number of rows (height), based on x/z scaling factor
  psf_im_height_up = int(im_s) # match number of rows
  psf_im_width_up = int(im_s*(psf_im_width/psf_im_height))
  psf_upscale_dim = (psf_im_width_up,psf_im_height_up)
  psf_im_up = np.zeros((psf_im_height_up,psf_im_width_up,3))
  for i in range(psf_im.shape[2]):
    psf_im_up[:,:,i] = cv2.resize(psf_im[:,:,i],psf_upscale_dim,interpolation = cv2.INTER_NEAREST)
  # construct final image
  white_space = np.ones((mip_boxed.shape[0], int(mip_boxed.shape[0]/50), 3))
  tot_im = np.hstack((mip_boxed, white_space, psf_im_up/255))
  plt.imshow(tot_im)
  plt.axis('off')
  # plt.title('PSF from: ' + fold + ' at coordinates ' + str(x) + "," + str(y))
  fig_path = gdr_blob_path[:gdr_blob_path.rfind('/')+1] + str(y) + "_" + str(x) + "_psf_vis.png"
  plt.savefig(fig_path, bbox_inches='tight', dpi=1000)
  plt.close()

In [ ]:
def addBoundingBox(I,x,y,r,extension=2): # thickness is 2
  if len(I.shape) == 2:
    ny, nx = I.shape
    I = np.dstack([I, I, I])
  else:
    ny, nx, nc = I.shape
  x_min = max(x - r - extension,0)
  y_min = max(y - r - extension,0)
  x_max = min(x + r + extension,nx-1)
  y_max = min(y + r + extension,ny-1)
  I[y_min-3:y_min+3,x_min:x_max+1,:] = [1, 0, 0]
  I[y_max-3:y_max+3,x_min:x_max+1,:] = [1, 0, 0]
  I[y_min:y_max+1,x_min-3:x_min+3,:] = [1, 0, 0]
  I[y_min:y_max+1,x_max-3:x_max+3,:] = [1, 0, 0]
  return I

In [ ]:
# save random PSF visualizations (blobs) from each objectives folder within the parent folder to the drive folder
# gs_folder_path: path to the parent folder in GCS which is directly above the folders for each objective lens configuration (each objective folder should contain the z-stack and blobs folder, created above)
# dr_save_path: path in Google Drive to which the randomly selected blobs will be saved
# num_blobs: number of blobs to select from each folder of blobs 
def save_rand_blobs(gs_folder_path, dr_save_path, num_blobs=10):
  obj_folds = !gsutil ls "{gs_folder_path}"
  obj_folds = [obj_folds[9]]
  for obj_fold in obj_folds: # for each objective folder (containing a z-stack for a given objective and its blobs folder)
    obj_fold_dr = obj_fold[:-1]
    obj_fold_dr = obj_fold_dr[obj_fold_dr.rfind('/')+1:]
    # # create equivalent folder in google drive if necessary
    # if not os.path.exists(dr_save_path + '/' + obj_fold_dr + '/0/blobs'): # create folder in drive that blobs will be saved to
    #   os.mkdir(dr_save_path + '/' + obj_fold_dr)
    #   os.mkdir(dr_save_path + '/' + obj_fold_dr + '/0/')
    #   os.mkdir(dr_save_path + '/' + obj_fold_dr + '/0/blobs/')
    #   print("made directory: " + dr_save_path + '/' + obj_fold_dr + '/0/blobs/')
    blobs = !gsutil ls "{obj_fold + "0/blobs"}"
    # select num_blobs blobs to save
    blobs_to_save = random.sample(blobs, num_blobs)
    for gcs_blob_path in blobs_to_save:
      bp = gcs_blob_path
      for i in range(4):
        bp = bp[bp.find('/')+1:]
      gdr_blob_path = dr_save_path + "/" + bp
      save_blob_local_and_global(gcs_blob_path,gdr_blob_path,mip_dict,(3/0.1665))

In [ ]:
x = !gsutil ls "{gs_folder_path_2}"
x[9]

'gs://octopi-malaria-data/20210310_objectives_characterizations_with_beads/beads #3/40x 0.6 0.17 coverslip, dz = 3 um, N = 16_2021-03-10 21-33-49.642710/'

In [ ]:
save_rand_blobs(gs_folder_path_2, dr_save_path)

Copying gs://octopi-malaria-data/20210310_objectives_characterizations_with_beads/beads #3/40x 0.6 0.17 coverslip, dz = 3 um, N = 16_2021-03-10 21-33-49.642710/0/blobs/2477_752.png...
- [1 files][  781.0 B/  781.0 B]                                                
Operation completed over 1 objects/781.0 B.                                      
2477,752
Copying gs://octopi-malaria-data/20210310_objectives_characterizations_with_beads/beads #3/40x 0.6 0.17 coverslip, dz = 3 um, N = 16_2021-03-10 21-33-49.642710/0/blobs/1702_749.png...
/ [1 files][  817.0 B/  817.0 B]                                                
Operation completed over 1 objects/817.0 B.                                      
1702,749
Copying gs://octopi-malaria-data/20210310_objectives_characterizations_with_beads/beads #3/40x 0.6 0.17 coverslip, dz = 3 um, N = 16_2021-03-10 21-33-49.642710/0/blobs/458_1162.png...
/ [1 files][  788.0 B/  788.0 B]                                                
Operation completed ove